In [1]:
import ee
ee.Initialize()
from s1flood import calc_basemean, calc_basesd, calc_zscore
import geemap

In [2]:
geometry = ee.Geometry.Polygon(
        [[[93.28456780903605, 24.071501387947965],
          [93.28456780903605, 22.22296242611564],
          [95.57521722309855, 22.22296242611564],
          [95.57521722309855, 24.071501387947965]]]
)

In [3]:
targdate = "2017-09-19"

filters = [
  ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
  ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
  ee.Filter.equals("instrumentMode", "IW"),
  ee.Filter.geometry(geometry),
  ee.Filter.date('2015-01-01', ee.Date(targdate).advance(1, 'day'))
  ]

In [4]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD").filter(filters)

In [5]:
basestart = "2017-06-01"
baseend = "2017-09-01"

meanvv = calc_basemean(s1, basestart, baseend, 'IW', 'DESCENDING').select('VV')
sdvv = calc_basesd(s1, basestart, baseend, 'IW', 'DESCENDING').select('VV')
zvv = calc_zscore(s1, basestart, baseend, 'IW', 'DESCENDING').select('VV')

In [6]:
Map = geemap.Map()
Map.setCenter(94.0953, 23.1513, 11)

In [7]:
Map.addLayer(s1.select('VV'), {'min': -25, 'max': 0}, 'VV Backscatter (dB); 2017-09-19')
Map.addLayer(meanvv, {'min': -25, 'max': 0}, 'Baseline mean VV backscatter (dB)')

sdpalette = ['#ffffff','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d']
Map.addLayer(sdvv, {'min': 2, 'max': 6, 'palette': sdpalette}, 'Baseline s.d. VV backscatter (dB)')

zpalette = ['#b2182b','#ef8a62','#fddbc7','#f7f7f7','#d1e5f0','#67a9cf','#2166ac']
Map.addLayer(zvv, {'min': -10, 'max': 10, 'palette': zpalette}, 'VV Z-score; 2017-09-19')

In [8]:
Map

Map(center=[23.1513, 94.0953], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…